In [1]:
# get openai embeddings
import pyterrier as pt
import os
from dotenv import load_dotenv
from dotenv import dotenv_values

from openai import OpenAI
from tqdm.auto import tqdm
import pandas as pd
tqdm.pandas()



from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from cord19_plus.data_model.model import Table
from cord19_plus.data_model.database_setup import setup_engine_session_alt
from cord19_plus.data_model.embeddings import Embedding, Base

from retrieval_utils import transform_to_catch_all, get_embedding,get_emb_df, populate_embs_from_result,transform_to_catch_all, num_tokens_from_string

load_dotenv("/workspaces/CORD19_Plus/.env")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
db_vals = dotenv_values("/workspaces/CORD19_Plus/.env")
client = OpenAI()

if not pt.started():
    pt.init()

import logging
logging.disable(logging.FATAL + 1)

/home/vscode/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_1233504/2578765857.py:29: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():
Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]
/tmp/ipykernel_1233504/2578765857.py:30: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [2]:
engine = create_engine(f"postgresql+psycopg2://{db_vals['USER']}:{db_vals['PASSWORD']}@{db_vals['ADDRESS']}:{db_vals['PORT']}/{db_vals['DB_FINAL']}", echo=False)
session = Session(engine)

result = session.query(Table)
result_dict = [{"docno" : str(e.ir_tab_id), "ir_id" : e.ir_id, "table_name" : e.table_name, "header": e.header, "content" : e.content, "caption" : e.caption, "references" : e.references} for e in result]

In [3]:
# generate embeddings for all entries in the 'Table' table
# populate_embs_from_result(result_dict)

26100 Tables already processed
3703 Tables to process


Overall Progress: 100%|██████████| 38/38 [06:54<00:00, 10.90s/it]

Processing and insertion complete.


In [6]:
#takes about 1 minute
emb_df = get_emb_df()
emb_df.to_json("/workspaces/CORD19_Plus/retrieval/table_embeddings.json")

In [ ]:
#generate topic embeddings
dataset = pt.get_dataset('irds:cord19/fulltext/trec-covid')

topics_t = dataset.get_topics('title')
topics_d = dataset.get_topics('description')
topics_n = dataset.get_topics('narrative')

topics_all = dataset.get_topics('title')

topics_all['query'] = topics_all.apply(lambda row: f"{topics_t.loc[int(row['qid'])-1]['query']}. {topics_d.loc[int(row['qid'])-1]['query']}. {topics_n.loc[int(row['qid'])-1]['query']}", axis=1)

In [ ]:
topic_embs = []

with tqdm(total=len(topics_all)) as pbar:
    for _, row in tqdm(topics_all.iterrows()):
        qid = row['qid']
        query = row['query']

        topic_embs.append({'qid' : qid, 'emb' : get_embedding(query)})
        pbar.update()

In [ ]:
df_queries = pd.DataFrame(topic_embs)
df_queries.to_json("/workspaces/CORD19_Plus/retrieval/query_embeddings.json", index=False)

In [ ]:
df_queries = pd.read_json("/workspaces/CORD19_Plus/retrieval/query_embeddings.json")